In [6]:
import math
import keras
import time
import random
import numpy as np
from PIL import Image
from keras.models import Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Input,BatchNormalization,MaxPooling2D,Conv2DTranspose,concatenate 

In [7]:
# Read data
size_pat=64 # pixel number of generated data in one dimension
max_pixel_num=333 # each image is 333 x 333 pixels
sample_1d = int(max_pixel_num/size_pat) # each image's sample number in one dimension
sample_num_per_img = int(math.pow(sample_1d,2))
print("Every image produces",sample_num_per_img,"non-repeating training images.")
class_to_train = 3 # [1,9]

least = 1
largest = 26
print("Using images from",least,"to",largest,".")

# Generating data from image number "least" to "largest"
total_sample_num = sample_num_per_img*(largest - least + 1)
patch=np.ones((total_sample_num,size_pat,size_pat,6),float)
label=np.zeros((total_sample_num,size_pat,size_pat,1),int)
class_count = np.zeros(9,dtype=int)
for i in range(least,largest+1):
    s = 'I:/cis731projdataset/Image'+str(i)+'_allData.csv'
    mydata = np.genfromtxt(s, delimiter=',')
    for rs in range(sample_1d):# sample image row index
        for cs in range(sample_1d):# sample image column index
            p_index = (i-least)*sample_num_per_img+cs*sample_1d+rs
            for ri in range(size_pat):# image pixel row index
                for ci in range(size_pat):# image pixel column index
                    data_index = (ci+cs*size_pat)*max_pixel_num+(rs*size_pat+ri)
                    patch[p_index,ri,ci,:] = mydata[data_index,0:6]
                    if(mydata[data_index,6] == 10):
                        print(i)
                        print(int(mydata[data_index,6]))
                    class_count[int(mydata[data_index,6])-1] += 1
                    if mydata[data_index,6]==class_to_train:
                        label[p_index,ri,ci,0]=1
print("patch shape:", patch.shape)
print("label shape:", label.shape )
print('Class count:')
for l in range(len(class_count)):
    print("class",l+1," count",class_count[l])

Every image produces 25 non-repeating training images.
Using images from 1 to 26 .
patch shape: (650, 64, 64, 6)
label shape: (650, 64, 64, 1)
Class count:
class 1  count 29704
class 2  count 161965
class 3  count 43262
class 4  count 12705
class 5  count 3725
class 6  count 924669
class 7  count 528721
class 8  count 851917
class 9  count 105732


In [8]:
# Allocate train set and test set
fraction = 0.80
train_num = int(total_sample_num * fraction)
test_num = total_sample_num - train_num

sample_index = random.sample(range(0,total_sample_num),total_sample_num)
train_index = sample_index[:train_num]
test_index = sample_index[train_num:]

train_x=np.zeros((train_num,size_pat,size_pat,6),float)
train_y=np.zeros((train_num,size_pat,size_pat,1),int)
for i in range(train_num):
    train_x[i] = patch[train_index[i]]
    train_y[i] = label[train_index[i]]
print("Train set shape:",train_x.shape,"and",train_y.shape)

test_x=np.zeros((test_num,size_pat,size_pat,6),float)
test_y=np.zeros((test_num,size_pat,size_pat,1),int)
for i in range(test_num):
    test_x[i] = patch[test_index[i]]
    test_y[i] = label[test_index[i]]
print("Test set shape:",test_x.shape,"and",test_y.shape)
# train_x=patch[0:600]
# train_y=label[0:600]
# test_x=patch[600:]
# test_y=label[600:]

Train set shape: (520, 64, 64, 6) and (520, 64, 64, 1)
Test set shape: (130, 64, 64, 6) and (130, 64, 64, 1)


In [9]:
# Build model

# parameters:
num_channel = 64 # if increase this, network reaches higher training accuracy faster and may overfitting
size_kernel = (3,3)

# The input tensor
inputs = Input(shape=(size_pat,size_pat,6))
print("input shape: ",np.shape(inputs))

#layer1 64x64x6 -> 64x64x num_channel
layer1 = Conv2D(num_channel, (3, 3),strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(inputs)


#layer2 64x64x num_channel -> 32x32x num_channel
layer2 = BatchNormalization(axis=-1, momentum=0.01)(layer1)
layer2_c = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer2)

layer2 = BatchNormalization(axis=-1, momentum=0.01)(layer2_c)
layer2 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer2)
layer2 = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(layer2)
#9

#layer3 32x32x num_channel -> 16x16x num_channel
layer3 = BatchNormalization(axis=-1, momentum=0.01)(layer2)
layer3 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer3)

layer3 = BatchNormalization(axis=-1, momentum=0.01)(layer3)
layer3_c = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer3)

layer3 = BatchNormalization(axis=-1, momentum=0.01)(layer3_c)
layer3 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer3)

layer3 = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(layer3)
#19

#layer4 16x16x num_channel -> 8x8x num_channel
# layer4 = BatchNormalization(axis=-1, momentum=0.01)(layer3)
# layer4 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer4)

# layer4 = BatchNormalization(axis=-1, momentum=0.01)(layer4)
# layer4_c = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer4)

# layer4 = BatchNormalization(axis=-1, momentum=0.01)(layer4_c)
# layer4 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer4)

# layer4 = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(layer4)


#layer4_1 8x8x num_channel -> 4x4x num_channel
# layer4_1 = BatchNormalization(axis=-1, momentum=0.01)(layer4)
# layer4_1 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='elu')(layer4_1)

# layer4_1 = BatchNormalization(axis=-1, momentum=0.01)(layer4_1)
# layer4_1_c = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='elu')(layer4_1)

# layer4_1 = BatchNormalization(axis=-1, momentum=0.01)(layer4_1_c)
# layer4_1 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='elu')(layer4_1)

# layer4_1 = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(layer4_1)


#layer5_0 4x4x num_channel -> 8x8x num_channel
# layer5_0 = BatchNormalization(axis=-1, momentum=0.01)(layer4_1) #layer4_1 if 64 -> 4
# layer5_0 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='elu')(layer5_0)

# layer5_0 = BatchNormalization(axis=-1, momentum=0.01)(layer5_0)
# layer5_0 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='elu')(layer5_0)

# layer5_0 = BatchNormalization(axis=-1, momentum=0.01)(layer5_0)
# layer5_0 = Conv2DTranspose(num_channel, size_kernel,strides=(2,2),padding='same',data_format='channels_last',activation='elu')(layer5_0)



#layer5 concat and 8x8x num_channel -> 16x16x num_channel
#layer5 = concatenate([layer4_1_c,layer5_0],axis=-1) #comment this if 64 -> 8

# layer5 = BatchNormalization(axis=-1, momentum=0.01)(layer4) #layer 4 if 64 -> 8
# layer5 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer5)

# layer5 = BatchNormalization(axis=-1, momentum=0.01)(layer5)
# layer5 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer5)

# layer5 = BatchNormalization(axis=-1, momentum=0.01)(layer5)
# layer5 = Conv2DTranspose(num_channel, size_kernel,strides=(2,2),padding='same',data_format='channels_last',activation='relu')(layer5)


#layer6 concat and 16x16x num_channel -> 32x32x num_channel
#layer6 = concatenate([layer4_c,layer5],axis=-1)

layer6 = BatchNormalization(axis=-1, momentum=0.01)(layer3)
layer6 = Conv2D(int(num_channel*1.5), size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer6)

layer6 = BatchNormalization(axis=-1, momentum=0.01)(layer6)
layer6 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer6)

layer6 = BatchNormalization(axis=-1, momentum=0.01)(layer6)
layer6 = Conv2DTranspose(num_channel, size_kernel,strides=(2,2),padding='same',data_format='channels_last',activation='relu')(layer6)
#28

#layer7 concat and 32x32x num_channel -> 64x64x num_channel
layer7 = concatenate([layer3_c,layer6],axis=-1)

layer7 = BatchNormalization(axis=-1, momentum=0.01)(layer2) #
layer7 = Conv2D(int(num_channel*1.5), size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer7)

layer7 = BatchNormalization(axis=-1, momentum=0.01)(layer7)
layer7 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer7)

layer7 = BatchNormalization(axis=-1, momentum=0.01)(layer7)
layer7 = Conv2DTranspose(num_channel, size_kernel,strides=(2,2),padding='same',data_format='channels_last',activation='relu')(layer7)
#38

#output layer8 concat and convout
layer8 = concatenate([layer2_c,layer7],axis=-1)

layer8 = BatchNormalization(axis=-1, momentum=0.01)(layer8)
layer8 = Conv2D(int(num_channel*1.5), size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer8)

layer8 = BatchNormalization(axis=-1, momentum=0.01)(layer8)
layer8 = Conv2D(num_channel, size_kernel,strides=(1, 1),padding='same',data_format='channels_last',activation='relu')(layer8)

layer8 = Conv2D(1, (1, 1),strides=(1,1),padding='same',data_format='channels_last',activation='sigmoid')(layer8)
#46
model = Model(inputs = inputs, outputs = layer8)

opt = keras.optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

input shape:  (?, 64, 64, 6)


In [10]:
# start training
print("Start running at: ", time.strftime("%Y-%m-%d %H:%M:%S"))
start = time.clock()
model.fit(train_x, train_y,batch_size=32, validation_split=0.3, epochs=100, verbose=1)

Start running at:  2017-12-09 04:52:31
Train on 364 samples, validate on 156 samples
Epoch 1/100
364/364 [==============================] - 22s - loss: 0.3230 - acc: 0.9186 - val_loss: 0.1891 - val_acc: 0.9719
Epoch 2/100
364/364 [==============================] - 11s - loss: 0.1350 - acc: 0.9831 - val_loss: 0.1009 - val_acc: 0.9827
Epoch 3/100
288/364 [======================>.......] - ETA: 2s - loss: 0.0852 - acc: 0.9849

KeyboardInterrupt: 

In [ ]:
prediction=model.predict(test_x)
elapsed = (time.clock() - start)
print("Time used(s):",elapsed)
eva = model.evaluate(test_x, test_y, verbose=0)
print("Accuracy on test set: ",eva[1])

In [ ]:
# Function for producing image
# def produce_img(name,data,boundary):
#     image_data=[0 for i in range(size_pat * size_pat)]
#     for i in range(len(data)):
#         for j in range(len(data[0])):
#             if (data[i][j][0] > boundary):
#                 image_data[i*size_pat+j] = 1
#     img = Image.new('1', (size_pat, size_pat))
#     img.putdata(image_data)
#     img.save(name)
    
# Function for producing image
def produce_img(name,data):
    image_data=[0 for i in range(size_pat * size_pat)]
    for i in range(len(data)):
        for j in range(len(data[0])):
            image_data[i*size_pat+j] = data[i][j][0] * 256
    img = Image.new('L', (size_pat, size_pat))
    img.putdata(image_data)
    img.save(name)
    
# from PIL import Image
# import random
# image_data=[random.randint(0,60) for i in range(128 * 128)]
# img = Image.new('L', (128, 128))
# img.putdata(image_data)
# img.save('11.jpg')
    

# Print out miss count and right count for 1s
def infor(prediction,label):
    all_count = 0
    miss_count = 0
    right_count = 0
    not_good_count = 0
    for i in range(len(prediction)):
        for j in range(len(prediction[0])):
            if label[i][j] == 1:
                all_count += 1
                if prediction[i][j] >= 0.9:
                    right_count += 1
                elif prediction[i][j] >= 0.4:
                    not_good_count += 1
                else:
                    miss_count +=1
    print('Number of 1s:{0:^7}, right count:{1:^7}, not good count:{2:^7}, miss count:{3:^7}'.
          format(all_count,right_count,not_good_count,miss_count))

In [ ]:
for i in range(50):
    sl = 'label_'+str(i)+'.jpg'
    sp = 'prediction_'+str(i)+'.jpg'
    produce_img(sl,test_y[i])
    produce_img(sp,prediction[i])

In [ ]:
for i in range(len(test_x)):
    infor(prediction[i],test_y[i])

In [ ]:
train_test_x = train_x[50:100]
train_test_y=train_y[50:100]
eva = model.evaluate(train_test_x, train_test_y, verbose=0)
print("Accuracy on part of train set: ",eva[1])
prediction=model.predict(train_test_x)
for i in range(len(train_test_x)):
    sl = 'testlabel_'+str(i)+'.jpg'
    sp = 'test_prediction_'+str(i)+'.jpg'
    produce_img(sl,train_test_y[i])
    produce_img(sp,prediction[i])